In [10]:
import pandas as pd
import numpy as np
import os

In [12]:
df= pd.read_csv('data_preprocessed.csv')
df

,image,Pid,Uid,Category,Subcategory,Concept,combined_text,ispro,canbuypro,photo_count,...,timezone_offset,groups,group_impact,group_weight,year,month,day,hour,dayofweek,label
0,images/385070026.jpg,775,59@N75,Fashion,Fashion,glam,Luis Drayton - Edinburgh shoot #6 rock punk tr...,1,0,8.828934,...,1,0.026014,0.012244,0.026187,2015,10,28,7,2,11.18
1,images/943750056.jpg,1075,1@N18,Travel&Active&Sports,Soccer,fifa,Arena da Barra - Arena HSBC - Arena do PAN #...,1,0,10.376705,...,0,0.004591,0.004978,0.003310,2016,2,9,2,1,15.15
2,images/3246928439.jpg,4890,351@N64,Entertainment,Movies,cinema,MARILYN 2015 old cinema beauty marilyn photosh...,0,0,5.624018,...,-3,0.000000,0.000000,0.000000,2015,6,1,0,0,10.99
3,images/3538960847.jpg,7079,1617@N40,Food,Drinks,thirsty,CAMELS01 hot sahara animal animals desert bott...,1,0,10.376705,...,0,0.000765,0.003345,0.008532,2015,3,7,16,5,11.16
4,images/3544037429.jpg,7111,1617@N40,Travel&Active&Sports,Gymnastics,flip,INDIA0457/ MY CHRISTMAS 2015 CARD......... str...,1,0,10.376705,...,0,0.000765,0.006269,0.000861,2015,3,21,5,5,12.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251290,images/27597418605.jpg,1118328,62225@N20,Animal,Insects,creature,Mushrooms york travel sky urban sun france col...,1,0,9.024252,...,12,0.006121,0.148169,0.160765,2015,6,6,7,5,4.00
251291,images/27597517425.jpg,1118334,108435@N35,Social&People,Selfies,portrait,Evie portrait cute animal cat furry panasonic ...,0,0,6.687109,...,-6,0.006121,0.036871,0.039748,2015,8,15,9,5,1.00
251292,images/27597606545.jpg,1118342,84730@N71,Travel&Active&Sports,Cars,exotic,BSSLS+R8GT!! car exotic german malaysia kualal...,0,0,6.354370,...,0,0.000000,0.000000,0.000000,2015,10,10,7,5,2.00
251293,images/27597637235.jpg,1118344,9542@N37,Whether&Season,Cloudy,cloud,C to A1 city newzealand christchurch cloud bui...,1,0,8.877382,...,10,0.000765,0.086168,0.063232,2015,9,17,3,3,6.89


In [14]:
import os
os.getcwd()

'd:\\old\\data255_deeplearning_team2\\data\\raw'

In [16]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image

#Load CLIP model
model = SentenceTransformer('clip-ViT-B-32')

#Encode an image:
img_emb = model.encode(Image.open('images/27592642565.jpg'))

#Encode text descriptions
text_emb = model.encode(['Two dogs in the snow', 'A cat on a table', 'A picture of London at night', 'human', 'girl', 'water', 'ocean', 'bridge'])

#Compute cosine similarities 
cos_scores = util.cos_sim(img_emb, text_emb)
print(cos_scores)


tensor([[0.1344, 0.1827, 0.1477, 0.2269, 0.2421, 0.2189, 0.2296, 0.2227]])


In [17]:
img_emb.shape

(512,)

In [53]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader, Dataset

# Check if GPU is available and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Initialize ResNet-50 model
# model = models.resnet50(pretrained=True).to(device)
# model.eval()

# Load MobileNetV3 Small
model = models.mobilenet_v3_small(pretrained=True).to(device)
model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Custom dataset class
class ImageDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        try:
            image = Image.open(image_path).convert('RGB')
            image = transform(image)
        except FileNotFoundError:
            print(f"File not found: {image_path}")
            # Return a zero tensor for missing images
            image = torch.zeros(3, 224, 224)  # Adjust the size as needed

        return image

c:\Users\31405.ISBDOMAIN1\AppData\Local\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\31405.ISBDOMAIN1\AppData\Local\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [54]:
# Create dataset and data loader
image_paths = df['image'].tolist()  # assuming 'image' column contains the paths
dataset = ImageDataset(image_paths)
loader = DataLoader(dataset, batch_size=64, shuffle=False)  # adjust batch size as needed

In [55]:
from tqdm import tqdm
# Function to process images in batches and generate embeddings
def generate_embeddings(loader, model, device):
    with torch.no_grad():
        embeddings = []
        for batch in tqdm(loader, desc="Processing Images"):
            try:
                batch = batch.to(device)
                batch_embeddings = model(batch)
                embeddings.append(batch_embeddings.cpu().numpy())
            except Exception as e:
                print(f"Error processing batch: {e}")
                # Append zero array of shape (1000,) for each image in the batch
                zero_embedding = np.zeros((batch.shape[0], 1000))
                embeddings.append(zero_embedding)
        return np.vstack(embeddings)

In [57]:
len(image_paths)

251295

In [58]:
# Generate image embeddings
image_embeddings = generate_embeddings(loader, model, device)

Processing Images: 100%|██████████| 3927/3927 [1:09:28<00:00,  1.06s/it]


In [52]:
device

device(type='cpu')

In [60]:
# Save the embeddings to a NumPy file
np.save('image_embeddings.npy', image_embeddings)

In [61]:
image_embeddings[0].shape

(1000,)

In [62]:
len(image_embeddings)

251295

In [63]:
# Assuming 'data['image']' contains image paths
first_image_path = df['image'].iloc[0]

# Process the first image and generate embedding
first_image_embedding = model(transform(Image.open(first_image_path).convert('RGB')).unsqueeze(0).to(device))
first_image_embedding = first_image_embedding.cpu().detach().numpy()
first_image_embedding

array([[-7.72040749e+00, -4.08484793e+00, -2.44221544e+00,
        -4.61581755e+00, -4.97474766e+00, -4.27396917e+00,
        -5.22054482e+00, -1.01205337e+00, -2.08745289e+00,
        -3.91398454e+00, -7.19348478e+00, -4.54616737e+00,
        -3.65873003e+00, -5.05676079e+00, -5.25836897e+00,
        -6.20901632e+00, -4.87644815e+00, -5.72395802e+00,
        -3.64491749e+00, -2.66971874e+00, -6.63237095e+00,
        -6.31971121e+00, -3.12121034e+00, -2.31668043e+00,
        -4.61210632e+00, -5.59682226e+00, -8.34546089e+00,
        -7.35157251e+00, -6.29106903e+00, -5.49883366e+00,
        -7.20295334e+00, -7.06838799e+00, -5.67575073e+00,
        -7.94529533e+00, -6.04913950e+00, -6.72351170e+00,
        -6.24419975e+00, -8.36747265e+00, -7.85525846e+00,
        -8.70736599e+00, -8.74412632e+00, -9.19083500e+00,
        -9.41989613e+00, -6.70887136e+00, -8.58362770e+00,
        -7.14226055e+00, -7.80471611e+00, -1.17925329e+01,
        -7.92870045e+00, -4.68002796e+00, -6.08907843e+0

In [64]:
image_embeddings[0].shape

(1000,)

In [65]:
# Reshape the embeddings to 2D arrays (1 sample, many features)
first_image_embedding_reshaped = first_image_embedding.reshape(1, -1)
first_in_embeddings_reshaped = image_embeddings[0].reshape(1, -1)

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

# Compare the first embedding
cosine_sim = cosine_similarity(first_image_embedding_reshaped, first_in_embeddings_reshaped)[0][0]

print(f"Cosine Similarity: {cosine_sim}")

Cosine Similarity: 1.0000001192092896
